In [1]:
using LinearAlgebra

include("./MPS.jl")
using .MPSforQuantum

## MPOの用意

$Z_1 Z_2 + Z_2 Z_3 + Z_3 Z_4$

$$
\left[\begin{array}{c}
    Z_1 & I_1 & I_1\\
\end{array}\right]
\left[\begin{array}{c}
    Z_2 & 0 & 0\\
    0 & Z_2 & 0\\
    0 & 0 & I_2\\
\end{array}\right]
\left[\begin{array}{c}
    I_3 & 0 & 0\\
    0 & Z_3 & 0\\
    0 & 0 & Z_3\\
\end{array}\right]
\left[\begin{array}{c}
    I_4\\
    I_4\\
    Z_4
\end{array}\right] \quad
$$

In [2]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))
O[4]

2×2×3 Array{Complex{Float64},3}:
[:, :, 1] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

[:, :, 2] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

[:, :, 3] =
 1.0+0.0im   0.0+0.0im
 0.0+0.0im  -1.0+0.0im

In [3]:
N = 4
eps = 1e-3
D = 10

for i in 1:16
C0 = zeros(ComplexF64, 2^N)
C0[i] = 1 # '0001'
mps = MPS(C0, eps)
# println(restore(mps, 4))
expc = expectation(mps, O)
println(expc)
end

Complex{Float64}[3.0 + 0.0im]
Complex{Float64}[1.0 + 0.0im]
Complex{Float64}[-1.0 + 0.0im]
Complex{Float64}[1.0 + 0.0im]
Complex{Float64}[-1.0 + 0.0im]
Complex{Float64}[-3.0 + 0.0im]
Complex{Float64}[-1.0 + 0.0im]
Complex{Float64}[1.0 + 0.0im]
Complex{Float64}[1.0 + 0.0im]
Complex{Float64}[-1.0 + 0.0im]
Complex{Float64}[-3.0 + 0.0im]
Complex{Float64}[-1.0 + 0.0im]
Complex{Float64}[1.0 + 0.0im]
Complex{Float64}[-1.0 + 0.0im]
Complex{Float64}[1.0 + 0.0im]
Complex{Float64}[3.0 + 0.0im]


## 初期状態ベクトルを適当に用意

In [5]:
N = 4
C0 = normalize!(rand(ComplexF64, 2^N))

D = 20
mps = MPS(C0, D, 'r') # convert to MPS
mps_size(mps) # print the size of MPS

expc = expectation(mps, O)
println("Initial Expectation Value: ", expc)

array 1's size: (1, 2)
array 2's size: (2, 4)
array 3's size: (4, 2)
array 4's size: (2, 1)
Num of parameters: 40
2^N: 16
Initial Expectation Value: Complex{Float64}[-0.48136571796827476 - 2.7755575615628914e-17im]


In [ ]:
function R_expression(mps, O, t)
    N_site = size(mps)[1]
    n_phys = 2
    contracted_sites = []
    for i = t+1:N_site
        if i==t+1
            a_len1 = size(mps[i-1][1])[2]
            a_len2 = size(mps[i][1])[2]
            b_len = size(O[i][1, 1, :])[1]
            arr = zeros(ComplexF64, a_len1, a_len2, b_len, b_len, a_len1, a_len2)
            for a1 = 1:a_len1, a2 = 1:a_len2, b1 = 1:b_len, b2 = 1:b_len, a1_ = 1:a_len1, a2_ = 1:a_len2
                for sigma1 = 1:n_phys, sigma2 = 1:n_phys
                    arr[a1, a2, b1, b2, a1_, a2_] += mps[i][sigma1]'[a2, a1] * O[i][sigma1, sigma2, b1, b2] * mps[i][sigma2][a1_, a2_]
                end
            end
        push!(contracted_sites, arr)
            
        elseif i==N_site
            # last step
            a_len1 = size(mps[i-1][1])[2]
            a_len2 = size(mps[i][1])[2]
            b_len2 = size(O[i][1, 1, :])[1]
            arr_0 = zeros(ComplexF64, n_phys, a_len1, b_len2, a_len2)
            for sigma2 = 1:n_phys, a1=1:a_len1, b2 = 1:b_len2, a2 = 1:a_len2
                for a1_0=1:a_len1
                    arr_0[sigma2, a1, b2, a2] += contracted_sites[i-t][a1, b2, a1_0] * mps[i][sigma2][a1_0, a2]
                end
            end

            arr_1 = zeros(ComplexF64, n_phys, a_len1, a_len2)
            for sigma1 = 1:n_phys, a1=1:a_len1, a2 = 1:a_len2
                for sigma2 = 1:n_phys, b2 = 1:b_len2
                    arr_1[sigma1, a1, a2] += O[i][sigma1, sigma2, b2] * arr_0[sigma2, a1, b2, a2]
                end
            end

            arr = zeros(ComplexF64, a_len2, a_len2)
            for a2 = 1:a_len2, a2 = 1:a_len2
                for sigma1 = 1:n_phys, a1=1:a_len1
                    arr[a2, a2] += mps[i][sigma1]'[a2, a1] * arr_1[sigma1, a1, a2]
                end
            end
        
        else
            a_len1 = size(mps[i-1][1])[2]
            a_len2 = size(mps[i][1])[2]
            b_len = size(O[i][1, 1, :, :])[1]
            arr = zeros(ComplexF64, a_len2, b_len, a_len2)
            for sigma2 = 1:n_phys, a1 = 1:a_len1, b1 = 1:b_len1, a2=1:a_len2
                for a0_0 = 1:a_len1
                    arr_0[sigma2, a1, b1, a2] += contracted_sites[i-t][a1, a2, b1, b2, a1_, a2_] * mps[i][sigma2][a0_0, a2]
                end
            end

            arr_1 = zeros(ComplexF64, n_phys, b_len2, a_len1, a_len2)
            for sigma1 = 1:n_phys, b2 = 1:b_len2, a1 = 1:a_len1, a2=1:a_len2
                for sigma2=1:n_phys, b1 = 1:b_len1
                    arr_1[sigma1, b2, a1, a2] += O[i][sigma1, sigma2, b1, b2] * arr_0[sigma2, a1, b1, a2]
                end
            end

            arr = zeros(ComplexF64, a_len2, b_len2, a_len2)
            for a2_=1:a_len2, b2 = 1:b_len2, a2=1:a_len2
                for sigma1 = 1:n_phys, a1 = 1:a_len1
                    arr[a2, b2, a2_] += mps[i][sigma1]'[a2, a1] * arr_1[sigma1, b2, a1, a2_]
                end
            end
        end
    end
end

In [6]:
using TensorOperations

In [9]:
A = pauliZ
B = pauliZ
C = zeros(2,2)
@tensor begin
    C[a,c] = A[a,b]*B[b,c]
end

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0